In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
from pyspark.sql.functions import col, to_date, count
from pyspark.sql.functions import from_unixtime

spark = SparkSession.builder \
    .appName("Test_listen_events") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/12/05 14:44:56 WARN Utils: Your hostname, DESKTOP-C1UKSNA resolves to a loopback address: 127.0.1.1; using 172.24.216.244 instead (on interface eth0)
24/12/05 14:44:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/05 14:44:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/05 14:44:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
data_path = "/mnt/c/Users/Jovan Bogoevski/StreamsSongs/processed_topics/listen_events"

data = spark.read.parquet(data_path)

data.printSchema()

data.show()

root
 |-- artist: string (nullable = true)
 |-- song: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- ts: long (nullable = true)
 |-- sessionId: integer (nullable = true)
 |-- auth: string (nullable = true)
 |-- level: string (nullable = true)
 |-- itemInSession: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- state: string (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- lon: double (nullable = true)
 |-- lat: double (nullable = true)
 |-- userId: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- registration: date (nullable = true)



+--------------------+--------------------+---------+-------------+---------+---------+-----+-------------+-----------+-----+-----+--------------------+----------+---------+------+-----------+---------+------+------------+
|              artist|                song| duration|           ts|sessionId|     auth|level|itemInSession|       city|  zip|state|           userAgent|       lon|      lat|userId|   lastName|firstName|gender|registration|
+--------------------+--------------------+---------+-------------+---------+---------+-----+-------------+-----------+-----+-----+--------------------+----------+---------+------+-----------+---------+------+------------+
|                MIKA|         Grace Kelly|188.08118|1609459241000|     1741|Logged In| paid|           85|   Verdigre|68783|   NE|Mozilla/5.0 (Maci...|-98.159234|42.618474|   187|      Smith|    Riley|     F|  2021-06-14|
|California Swag D...|Teach Me How To D...| 239.1767|1609459350000|     1729|Logged In| paid|          110|C

In [3]:
row_count = data.count()

# Print the row count
print(f"Total number of rows in listen events: {row_count}")

Total number of rows in listen events: 1188827


In [4]:
# Count the number of unique IDs in the 'userId' column
unique_id_count = data.select("userId").distinct().count()

# Print the unique ID count
print(f"Total number of unique IDs: {unique_id_count}")


Total number of unique IDs: 1000


In [5]:
unique_city_combinations = data.select(
    col("city").alias("city_name"),
    col("state"),
    col("zip").alias("zip_code"),
    col("lat").alias("latitude"),
    col("lon").alias("longitude")
).dropDuplicates().count()
print(f"Total number of unique city combinations: {unique_city_combinations}")

Total number of unique city combinations: 956


In [ ]:
unique_city_combinations = data.select(
    col("artist").alias("artist_name"),
    col("lat").alias("latitude"),
    col("lon").alias("longitude")
).dropDuplicates().count()
print(f"Total number of unique city combinations: {unique_city_combinations}")

In [6]:
unique_artists_in_source = data.select(
    col("artist").alias("artist_name")
).dropDuplicates(["artist_name"]).count()
print(f"Unique artists in source dataset: {unique_artists_in_source}")


Unique artists in source dataset: 26702


In [7]:
unique_songs_in_source = data.select(
    col("song").alias("song_name"),
    col("duration").alias("song_duration")
).dropDuplicates(["song_name", "song_duration"]).count()
print(f"Unique songs in source dataset: {unique_songs_in_source}")


Unique songs in source dataset: 144231


In [8]:
source_session_count = data.select("sessionId").distinct().count()
print(f"Unique sessions in source dataset: {source_session_count}")

Unique sessions in source dataset: 85147


In [10]:
source_unique_timestamps = data.select(
    from_unixtime(col("ts") / 1000).alias("timestamp")
).distinct().count()
print(f"Source unique timestamps: {source_unique_timestamps}")


Source unique timestamps: 1161431


In [11]:
source_unique_device = data.select("userAgent").distinct().count()
print(f"Source unique device: {source_unique_device}")

Source unique device: 83
